Imports

In [11]:
import datetime
import pandas as pd
from pandas import DataFrame
from openpyxl import load_workbook,worksheet,Workbook

Constants

In [12]:
MYDIR= r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files"
OUTPUT_DIR=r"C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files"

PNs to be Removed

In [13]:
list_of_pns = [
    96699290, 97775274, 96699299, 97775277, 96778078,
    97780992, 96699305, 96769184, 97778033, 96769190,
    96769205, 97778039, 96769256, 96896891, 96769259,
    96769271, 97780970, 96769280, 97780973
]

str_list = [str(numstr) for numstr in list_of_pns]

Justification Note

In [14]:
def add_removal_note():
    timeStamp = datetime.datetime.now().strftime("%m-%d-%Y")
    reason = input("Reason for removal: ")
    authority = input("Who authorized/requested this change? ")
    return timeStamp + " " + reason + " per " + authority


Geeting the First Row of the PSD

In [15]:
wb=load_workbook(r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\input files\Lbom-ES.xlsx',read_only=True)

In [16]:
def get_start_row(wb:Workbook,sheet_name:str,min_row:int=None,max_row:int=None,min_col:int=None,max_col:int=None):
    try:
        if not sheet_name in wb.sheetnames:
            raise Exception
        ws=wb[sheet_name]
    except:
        print('Sheet does not exist')
    min_col = min_col or 1
    min_row = min_row or 1
    max_col = max_col or ws.max_column
    max_row = max_row or ws.max_row
    for row in ws.iter_rows(min_row,max_row,min_col,max_col):
        if row[0].value=='[START]':
            return row[0].row-1

psd_startrow=get_start_row(wb,'Impeller',min_row=1,max_col=1)
psd_startcol=0
        
    

Grouping Function

In [17]:
def group_then_separate_by(psd_data:DataFrame,list_of_cols: list, pn_col: str)->tuple[DataFrame,DataFrame]:
    """list of cols are grouping categories. pn_col is the column that contains pns to find matches on."""
    groups = psd_data.groupby(list_of_cols) # Had to play around with this to get the right groupings

    df_list_to_remove = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later
    df_list_to_keep   = [] # will hold list of dataframes to remain in PSD. Will concatenate to 1 dataframe later

    # Iterates through each sub-group, checking if there is a pn that meets criteria for removal
    for _, frame in groups:
        if any(frame[pn_col].isin(str_list)):   # Finding matching partnumbers to remove
            df_list_to_remove.append(frame)     # Need to add this sub-group to a "removed dataframe"
        else:
            df_list_to_keep.append(frame)       # Need to retain this sub-group, add to a "keep dataframe"

    # Concatenating list of dfs to single dfs.
    removals = pd.concat(df_list_to_remove)
    keep = pd.concat(df_list_to_keep)
        
    return removals, keep

Iterate Through Directory

In [18]:
import os
import numpy as np
from typing import Callable,Any,ParamSpec,TypeVar,Concatenate
from _utils.file_ops import read_files_in_dir
from _utils.Dataframe_tools import get_df

P = ParamSpec('P')

######## START HERE #####
#Workbook contains Sheets and each file contains only one workbook therefore each file contains multiple sheets
#We want to make a list for any given file of the sheets we want to process 
#We can take a single sheet and process over it with the below function

# def process_sheet(sheet):
#     pass

#If we want to process multiple sheets within a file we need to loop over an iterable of the sheets we want and we can define this as a sepertate function.

def process_file(file,sheets):
    for sheet in sheets:
        process_sheet(sheet)

#I need to process each of the files in my directory 
def process_dir(dir):
    for file in dir:
        process_file(file)

files=read_files_in_dir(MYDIR)
sheetname='Impeller'


for file in files:
    removal_note=add_removal_note()
    data,fname=file
    psd_data=pd.read_excel(data,sheet_name=sheetname,header=psd_startrow-1,dtype={'BOM':str})
    original_psd_bottom_row = len(psd_data) + psd_startrow
    
    print("Opening file for updates: {}".format(os.path.basename(fname)))
    end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
    psd_data = psd_data.iloc[:end_row]
    group_by_columns = ["Model", "Price ID"]
    removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
    removals.loc[removals["Full Data"] == "[START]", "Full Data"] = ""
    new_row = pd.DataFrame({'ID': removal_note}, index =[0])
    removals = pd.concat([new_row, removals[:]]).reset_index()
    column_list = keep.columns
    removals = removals[column_list]
    removals.sort_values(by=['ID'], inplace=True)
    keep.loc[0,'Full Data'] = np.nan
    keep.sort_values(by=['ID'], inplace=True)
    keep.reset_index(drop=True, inplace=True)
    keep.loc[0,'Full Data'] = "[START]"
    num_rows = len(keep)                      
    keep.loc[num_rows,'Full Data']="[END]"
    append_location = original_psd_bottom_row - len(removals) 
    after_end_row = num_rows + psd_startrow + 2


#Assuming only one sheet for right now
# def before_change_data(dir,sheet_name:list[list[str]],is_same_note=True):
#     files=read_files_in_dir(dir)
#     for i,file in files:
#         sheet=[]
#         note=add_removal_note()
#         for sheet in sheet_name[i]:
#             sheet.append({'header':get_start_row(load_workbook(file[0],read_only=True),sheet_name=sheet,min_row=1,max_col=1)})
#             if not is_same_note:
#                 note=add_removal_note()
            
# before_change_data()


# a=list(before_change_data(MYDIR))


# def change_data(file:tuple[bytes,str,str],psd_data_func:Callable[Concatenate[P],DataFrame],psd_data_args:tuple[list,dict]):
#     removal_note=add_removal_note()
#     data,fname=file
#     psd_data=psd_data_func(data,*psd_data_args[0]) #will have to be called as a list or all have to be the same
#     original_psd_bottom_row = len(psd_data) + psd_startrow
#     print("Opening file for updates: {}".format(os.path.basename(fname)))
#     #...This is where the real processing begins...
#     end_row = psd_data[psd_data['Full Data'] == '[END]'].index.to_list()[0]
#     psd_data = psd_data.iloc[:end_row]
#     group_by_columns = ["Model", "Price ID"]
#     removals, keep = group_then_separate_by(psd_data,group_by_columns, "BOM")
#     removals.loc[removals["Full Data"] == "[START]", "Full Data"] = np.nan
#     new_row = pd.DataFrame({'ID': removal_note}, index =[0])
#     removals = pd.concat([new_row, removals[:]]).reset_index()
#     column_list = keep.columns
#     removals = removals[column_list]
#     removals.sort_values(by=['ID'], inplace=True)
#     keep.loc[0,'Full Data'] = np.nan
#     keep.sort_values(by=['ID'], inplace=True)
#     keep.reset_index(drop=True, inplace=True)
#     keep.loc[0,'Full Data'] = "[START]"
#     return keep,removals,fname



Opening file for updates: Lbom-ES.xlsx


Checking for Misses

In [19]:
if (len(keep[keep['BOM'].isin(str_list)])) == 0:
    print("Didn't miss any partnumbers. Good to go")
else:
    print("For some reason, the following entries were missed")
    print(keep[keep['BOM'].isin(str_list)])

Didn't miss any partnumbers. Good to go


Return a formatted file

In [10]:
from openpyxl.styles import PatternFill

outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'

wb = load_workbook(outputPath)
ws = wb['Impeller']
tabName = sheetname + " No Bronze"
wb.copy_worksheet(ws).title = tabName
ws_modified = wb[tabName]
ws_modified.delete_rows(after_end_row, len(removals)-1)
end_cell=ws.cell(row=after_end_row-1,column=1)
end_cell.fill=PatternFill(fill_type='solid',start_color="FFCC99",end_color="FFCC99")
for rows in ws_modified.iter_rows(min_row=append_location+2, max_row=append_location+2, min_col=1, max_col=40):
    for cell in rows:
        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
wb.save(outputPath)

In [20]:
# df=pd.read_excel(outputPath,'Impeller',header=None)
# header=df.loc[:4,:]
# rest=df.loc[5:,:]
# rest.reset_index(drop=True, inplace=True)
# rest.columns=rest.loc[0,:]
# rest=rest.drop(0)
from typing import overload,Literal,Sequence,Hashable,Iterable,Callable
from pandas._typing import DtypeArg,StorageOptions
from pandas.util._decorators import Appender


outputPath=r'C:\Projects\2022\Michaels_Code\grundfos-express-tools\bronze impeller removal\output files\Revised Lbom-ES.xlsx'




def _find_header_end(df:DataFrame)->int:
    for row in df.itertuples():
        if row[1]=='[START]':
            return int(row[0])


@overload
def _get_df(
    io,
    # sheet name is str or int -> DataFrame
    sheet_name: str | int,
    header: int | Sequence[int] | None = ...,
    names=...,
    index_col: int | Sequence[int] | None = ...,
    usecols=...,
    squeeze: bool | None = ...,
    dtype: DtypeArg | None = ...,
    engine: Literal["xlrd", "openpyxl", "odf", "pyxlsb"] | None = ...,
    converters=...,
    true_values: Iterable[Hashable] | None = ...,
    false_values: Iterable[Hashable] | None = ...,
    skiprows: Sequence[int] | int | Callable[[int], object] | None = ...,
    nrows: int | None = ...,
    na_values=...,
    keep_default_na: bool = ...,
    na_filter: bool = ...,
    verbose: bool = ...,
    parse_dates=...,
    date_parser=...,
    thousands: str | None = ...,
    decimal: str = ...,
    comment: str | None = ...,
    skipfooter: int = ...,
    convert_float: bool | None = ...,
    mangle_dupe_cols: bool = ...,
    storage_options: StorageOptions = ...,
) -> tuple[DataFrame,DataFrame,int]:
    ...


@Appender(pd.read_excel.__doc__,join='\n')
def _get_df(*args,**kwargs):
    """This extends the pandas read_excel method for our uses.
    """
    df=pd.read_excel(*args,**kwargs)
    header_size=_find_header_end(df)
    header=df.loc[:header_size-2,:]
    psd_df=df.loc[header_size-1:,:]
    psd_df.reset_index(drop=True,inplace=True)
    psd_df.columns=psd_df.loc[0,:]
    psd_df=psd_df.drop(0)
    original_length=len(df)
    return header,psd_df,original_length
    








    



        




Process Sheet

In [21]:
from _types._types import FilePath
from pandas._typing import FilePath, ReadBuffer
from typing import Union
from _utils.file_ops import read_files_in_dir

files=read_files_in_dir(MYDIR)
data,fname=next(files)

def process_sheet(file: tuple[Union[FilePath, ReadBuffer[bytes], bytes],str], sheet_name: str | list[str],removal_note:str|None|bool):
    """
    Params:
    data: Data comes from the binary of the file or the file path. 
    sheet_name: The name of the sheet to process.
    removal_note: Does the sheet have its own removal note or does it share one with the entire workbook (file). The default is set to it shares a note."""
    #Check if sheetname is a valid sheet in the file
    data,fname=file
    try:
        if not sheet_name in pd.ExcelFile(data).sheet_names:
            raise ValueError(f"Worksheet named '{sheet_name}' not found")
    except ValueError as err:
        print(err)
    #Need to get the header dataframe and the psd_dataframe
    header,psd_data,original_length=_get_df(data,sheet_name=sheet_name,header=None)
    psd_data.astype({"BOM":str},copy=False)
    if not bool(removal_note):
        removal_note=add_removal_note()
    return header
    












Making a New File

In [24]:

from _utils.excel_tools import write_excel_file
from _utils.file_ops import add_str_to_filename
from _utils.Dataframe_tools import write_df_to_excel

header=process_sheet(file,'Impeller','No')
tabName = sheetname + " No Bronze"

write_df_to_excel(keep,outputPath,tabName,startrow=psd_startrow-1,startcol=psd_startcol)
write_df_to_excel(removals,outputPath,tabName,header=False, startrow=append_location+1, startcol=psd_startcol)
write_df_to_excel(header,outputPath,tabName,header=False, startrow=0, startcol=0)